# Yolo-v1的pytorch实现

依照yolov1.cfg、yolov1原论文、[github上一份pytorch实现](https://github.com/ProgrammerZhujinming/YOLOv1/blob/main/YOLO_V1_Model.py)；

yolov1.cfg最后的fc层内容没太看懂；

module.py是方便fine-tune的版本；

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import sys
sys.path.append('../..')
import utils

/data/workspace/yuzijian/miniconda3/envs/d2l/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


测试填充函数：
- 输入的tensor：（batch，channel，height， width）
- 填充pad：[宽左，宽右，高上，高下]

In [2]:
x = torch.randn((1, 1, 4, 4))
x.shape, x

(torch.Size([1, 1, 4, 4]),
 tensor([[[[ 0.6121,  0.9437, -0.5037, -0.8590],
           [-0.2827,  1.1770,  0.0155, -0.1981],
           [ 1.6830, -0.6217, -0.2851, -0.6225],
           [-1.4199,  0.6816, -1.1158,  0.8052]]]]))

In [3]:
y = F.pad(x, pad=[1,2,3,4])
y.shape, y

(torch.Size([1, 1, 11, 7]),
 tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.6121,  0.9437, -0.5037, -0.8590,  0.0000,  0.0000],
           [ 0.0000, -0.2827,  1.1770,  0.0155, -0.1981,  0.0000,  0.0000],
           [ 0.0000,  1.6830, -0.6217, -0.2851, -0.6225,  0.0000,  0.0000],
           [ 0.0000, -1.4199,  0.6816, -1.1158,  0.8052,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]]))

In [4]:
f = nn.ZeroPad2d([1,2,3,4])
y = f(x)
y.shape, y

(torch.Size([1, 1, 11, 7]),
 tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.6121,  0.9437, -0.5037, -0.8590,  0.0000,  0.0000],
           [ 0.0000, -0.2827,  1.1770,  0.0155, -0.1981,  0.0000,  0.0000],
           [ 0.0000,  1.6830, -0.6217, -0.2851, -0.6225,  0.0000,  0.0000],
           [ 0.0000, -1.4199,  0.6816, -1.1158,  0.8052,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]]))

In [5]:
a = y[0, 0, 0, :4]
a

tensor([0., 0., 0., 0.])

虽然这个自适应padding形状的函数没用上

In [6]:
def compute_padding_shape(input_shape, kernel_size, stride):
    '''
    根据输入形状和步长，自动计算合适的padding_shape，以保持卷积过后，形状不变
    
    返回list[int]，[宽左，宽右，高上，高下]，为零填充的像素数
    '''
    padding=[]
    height, width = input_shape
    if isinstance(kernel_size, int):
        kernel_h, kernel_w = kernel_size, kernel_size
    else:
        kernel_h, kernel_w = kernel_size
    if isinstance(stride, int):
        stride_h, stride_w = stride, stride
    else:
        stride_h, stride_w = stride
    
    # 宽，(width - kernel_w + padding_w_left + padding_w_right) / stride_w + 1 == width
    padding.append(((width-1)*stride_w - width + kernel_w) / 2)             # 整除，取下整
    if ((width-1)*stride_w - width + kernel_w) % 2 == 0:
        padding.append(((width-1)*stride_w - width + kernel_w) / 2)
    else:
        padding.append(((width-1)*stride_w - width + kernel_w) / 2 + 1)
    # 高，也是类似
    padding.append(((height-1)*stride_h - height + kernel_h) / 2)
    if ((height-1)*stride_h - height + kernel_h) % 2 == 0:
        padding.append(((height-1)*stride_h - height + kernel_h) / 2)
    else:
        padding.append(((height-1)*stride_h - height + kernel_h) / 2 + 1)
    
    return padding

### 1、网络结构

网络中重复结构（卷积层）的构建函数，启发于VGG

In [7]:
def create_conv_layer(in_channels, out_channels, kernel_size, stride, padding, batch_normalize=1, activation='leaky'):
    '''
    构建yolov1中的卷积层, 形如pad->conv2d->batch_norm->activation function
    
    Parameters:
    - in_channels, out_channels, kernel_size, stride, padding: 同nn.Conv2d
    - batch_normalize: 是否要做批量归一化, 默认要做
    - activation: 激活函数种类, 默认LeakyReLU
    '''
    layers = []
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding))
    if batch_normalize:
        layers.append(nn.BatchNorm2d(out_channels))
    if activation=='leaky':
        layers.append(nn.LeakyReLU(inplace=True))
    return layers

网络结构如图，图源自原论文，网络输入形如(batch_size, channel, height, width)

<img src='./figs/yolov1_model.png' />

其中7x7代表将图片均匀划分成7x7个grid cell，每个grid cell负责判断是否有物体的中心点落在上面：

<img src='./figs/yolov1_grid.png' style='zoom:50%'>

每个grid对应一个长度为（5xB + class num）的向量：

- B为每个grid负责回归的边界框个数；
- 每个grid只能取B个中最好的一个作为预测值，即网络最多只能预测SxS个物体，论文中S=7，这也是YOLO-V1对小物体、密集物体效果不好的原因；
- 5：每个边界框$(x_c, y_c, w, h, c)$，中心点位置、高宽、置信度（该box负责预测物体）；
- class num：后面跟的ont hot向量，表示在该box负责预测物体的条件下，预测为类别i的条件概率（有疑问）；

In [8]:
class Yolo_v1(nn.Module):
    '''
    Yolo-v1的pytorch实现
    
    输入形状为448x448, B为每个grid预测的个数, Class_Num为分类数
    '''
    def __init__(self, B=2, Classes_Num=20):
        super().__init__()
        self.B = B
        self.Classes_Num = Classes_Num
        # 提前padding，保持各个块的卷积过后，形状不变；
        # in:3x448x448, out:64x112x112
        self.blk1 = nn.Sequential(
            *create_conv_layer(3, 64, kernel_size=7, stride=2, padding=3),  # stride=2的conv和pool都是减高宽的
            # nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=2),
            # nn.BatchNorm2d(64),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2, stride=2)
        )
        # in:64x112x112, out:192x56x56
        self.blk2 = nn.Sequential(
            *create_conv_layer(64, 192, kernel_size=3, stride=1, padding=1),
            # nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(192),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2, stride=2)
        )
        # in:192x56x56, out:512x28x28
        self.blk3 = nn.Sequential(
            *create_conv_layer(192, 128, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(128, 256, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(256, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, stride=2)
        )
        # in:512x28x28, out:1024x14x14
        self.blk4 = nn.Sequential(
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, stride=2)
        )
        # in:1024x14x14, out:1024x14x14
        self.blk4 = nn.Sequential(
            *create_conv_layer(1024, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1)
        )
        # in:1024x14x14, out:1024x7x7
        self.blk5 = nn.Sequential(
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=2, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1)
        )
        # in:7x7x1024, out:7x7x30
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7 * 1024, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 7 * 7 * (B*5 + Classes_Num)),
            nn.Sigmoid()        # 增加sigmoid函数是为了将输出全部映射到(0,1)之间，因为如果出现负数或太大的数，后续计算loss会很麻烦
        )

    def forward(self, x):
        # 类GoogLeNet
        # 也可换成resnet50，删除stage5后的全局池化和全连接层，增加一个带有空洞卷积的网络block（模仿detnet网络，增加感受野而不改变输出特征图大小）
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)
        # 后四层卷积
        x = self.blk5(x)
        # 为方便后处理，需要将输出构建成7x7x30，而不是30x7x7；为了符合直觉，需要做重排列将特征放在最后一维
        # (batch_size, channel, height, width) -> (batch_size, height, width, channel)
        x = x.permute(0, 2, 3, 1)
        # 展平+全连接层
        x = self.fc(x)
        # 重构
        x = x.view((-1, 7, 7, (self.B*5 + self.Classes_Num)))

    def initialize_params(self):
        '''网络参数初始化'''
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                m.bias.data.zero_()

疑问：(x,y,w,h,c)和p是否需要做softmax？？？
- 需要，我们用的结果是归一化的。

### 2、损失函数

Loss Function:

<img src='./figs/yolov1_loss.png' style='zoom:60%;' />

- 掩码：
  - $\mathbb{1}_{i}^{obj}$：有gt bbox的中心点落在grid $i$中；
  - $\mathbb{1}_{ij}^{obj}$：有gt bbox的中心点落在grid $i$中，且该grid回归的的B个bbox中，第$j$个bbox与该gt bbox的IoU最大；
  - $\mathbb{1}_{ij}^{noobj} == not \ (\mathbb{1}_{ij}^{obj})$；
- 第一项为中心点定位误差；
- 第二项为宽高定位误差：
  - 求根号可使小的bbox对误差更敏感；
- 第三、四项为confidence回归误差：
  - $\mathbb{1}_{ij}^{obj}==1$时（第三项），$\hat{C}_i$为该grid回归的第j个bbox，与中心点落在该grid的gt bbox的IoU；
  - $\mathbb{1}_{ij}^{obj}==0$时（第四项），$\hat{C}_i$为0；
  - $C_i$为预测值；
- 第五项为类别预测误差；
- 权重：
  - 物体检测问题，是一个典型的类别数目不均衡的问题。其中49个格点，含有物体的格点往往只有3、4个，其余全是不含有物体的格点。此时如果不采取点措施，那么物体检测的mAP不会太高，因为模型更倾向于不含有物体的格点。
  - $\lambda_{\mathbb{coord}}$：使网络更重视$(x_c,y_c,w,h)$的预测；
  - $\lambda_{noobj}$：对不负责预测obj的confidence loss，赋予小的权重；

In [9]:
class Yolo_v1_loss(nn.Module):
    '''
    Yolo-v1的损失函数计算的pytorch实现

    Parameters:
    - img_shape: _size_2_t, [h, w]
    - S: _size_2_t, 表示一张图片被划分成了[高x宽]个grid, 默认为7表示7x7
    - B: int, 一个grid回归多少个bbox
    - Class_num: int, 类别个数
    - l_coord、l_noobj: 对应两个lambda
    '''
    def __init__(self, img_shape=448, S=7, B=2, Class_num=20, l_coord=5, l_noobj=0.5):
        super().__init__()
        # 图片的形状
        if isinstance(img_shape, int):
            self.img_h = img_shape
            self.img_w = img_shape
        elif hasattr(img_shape, '__iter__'):
            self.img_h = img_shape[0]
            self.img_w = img_shape[1]
        # 网格的个数形状
        if isinstance(S, int):
            self.S_h = S
            self.S_w = S
        elif hasattr(S, '__iter__'):
            self.S_h = S[0]
            self.S_w = S[1]
        self.grid_h = self.img_h / self.S_h     # 网格的高度和宽度
        self.grid_w = self.img_w / self.S_w
        self.B = B
        self.Class_num = Class_num
        self.l_coord = l_coord
        self.l_noobj = l_noobj

    def forward(self, pred, gt):
        '''
        前向计算
        - pred:  (batch_size, S_h, S_w, (B)*5+Class_num), 未经过预处理, 仍然只是Yolo_v1输出的sigmoid
        - gt:    (batch_size, S_h, S_w, 5+Class_num), 已经经过预处理, 框成center形式(但中心点位置是相对于左上角的), 其中confidence代表1_i^{obj}, 类别成one-hot
        '''
        # 定义三个计算损失的变量 正样本定位损失 样本置信度损失 样本类别损失
        loss_coord = 0
        loss_confidence = 0
        loss_classes = 0
        iou_sum = 0
        object_num = 0

        mseLoss = nn.MSELoss()
        # pred的预处理
        for i in range(self.B):
            pred[:, :, :, i*5+0] = pred[:, :, :, i*5+0] * self.grid_w   # x_c相对于grid左上角的位置，最大不能超过一个grid
            pred[:, :, :, i*5+1] = pred[:, :, :, i*5+1] * self.grid_h   # y_c相对于grid左上角的位置
            pred[:, :, :, i*5+2] = pred[:, :, :, i*5+2] * self.img_w    # w，最大不能超过整张图片
            pred[:, :, :, i*5+3] = pred[:, :, :, i*5+3] * self.img_h    # h
        
        # loss计算
        # 其实也可以使用矩阵运算优化，但保险起见，还是用了循环
        for batch in range(len(pred)):
            for row in range(self.S_h):
                for col in range(self.S_w):
                    pred_ = pred[batch, row, col, :]
                    gt_ = gt[batch, row, col, :]
                    if gt_[4] == 0:
                        # [row, col]处的grid，没有gt框中心落在这，故B个框都是负样本，只需要加置信度loss
                        for i in range(self.B):
                            loss_confidence += self.l_noobj * torch.pow(pred_[i*5 + 4], 2)
                    else:
                        # [row, col]处的grid，有gt框中心落在这
                        object_num += 1
                        # 分别计算回归的B个框与gt的IoU
                        iou = torch.zeros(self.B)
                        gt_bbox = utils.bbox_center_to_corner(gt_[:4])
                        for i in range(self.B):
                            pred_bbox = utils.bbox_center_to_corner(pred_[i*5:(i*5+4)])
                            iou[i] = utils.IoU(pred_bbox, gt_bbox)
                        # 取IoU最大的成为正样本，其它B-1个成为负样本
                        val, idx = iou.topk(1)
                        # 定位loss
                        loss_coord += self.l_coord * (torch.pow(pred_[idx*5] - gt_[0], 2) + torch.pow(pred_[idx*5+1] - gt_[1], 2)) + \
                                      self.l_coord * (torch.pow(torch.sqrt(pred_[idx*5+2]) - torch.sqrt(gt_[2]), 2) + \
                                                      torch.pow(torch.sqrt(pred_[idx*5+3]) - torch.sqrt(gt_[3]), 2))
                        # 置信度loss
                        iou_sum += val
                        for i in range(self.B):
                            if i == idx:
                                # 正样本
                                loss_confidence += torch.pow(pred_[i*5 + 4] - val, 2)
                            else:
                                # 负样本
                                loss_confidence += self.l_noobj * torch.pow(pred_[i*5 + 4], 2)
                        # 类别预测loss
                        gt_class = gt_[5:]
                        pred_class = pred_[(self.B*5):]
                        loss_classes += mseLoss(gt_class, pred_class)
        loss_coord /= len(pred)
        loss_confidence /= len(pred)
        loss_classes /= len(pred)
        loss = loss_coord + loss_confidence + loss_classes
        return loss, loss_coord, loss_confidence, loss_classes, iou_sum, object_num

### 3、预测阶段后处理：非极大值抑制（NMS）

测试切片

In [10]:
a = torch.randint(1, 4, (2, 2, 2, 30))
a

tensor([[[[1, 1, 2, 1, 1, 3, 1, 1, 2, 3, 3, 1, 3, 1, 1, 1, 1, 2, 1, 3, 2, 1, 3,
           1, 2, 1, 1, 1, 1, 3],
          [1, 3, 3, 3, 1, 2, 1, 3, 2, 3, 2, 1, 2, 3, 2, 3, 3, 1, 2, 3, 2, 3, 2,
           1, 3, 1, 2, 2, 1, 1]],

         [[3, 1, 1, 3, 1, 1, 2, 3, 1, 1, 3, 3, 2, 3, 1, 1, 2, 2, 3, 3, 2, 3, 1,
           2, 2, 3, 2, 3, 3, 3],
          [1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 1, 3, 1, 2, 3, 2, 3, 1, 3,
           1, 3, 2, 1, 3, 1, 3]]],


        [[[1, 1, 1, 2, 2, 3, 3, 3, 2, 1, 1, 1, 1, 1, 3, 1, 2, 3, 2, 1, 3, 2, 3,
           3, 1, 1, 3, 1, 3, 1],
          [1, 3, 3, 3, 3, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 3, 3, 3,
           1, 1, 1, 2, 3, 1, 3]],

         [[2, 3, 1, 3, 3, 2, 2, 3, 3, 1, 3, 2, 2, 1, 2, 2, 1, 3, 3, 3, 3, 2, 1,
           2, 2, 3, 3, 1, 2, 3],
          [3, 3, 2, 2, 1, 1, 2, 1, 3, 2, 1, 3, 2, 3, 1, 1, 3, 1, 3, 2, 1, 1, 2,
           1, 3, 2, 3, 2, 1, 3]]]])

In [11]:
x, y, z = torch.split(a, [1, 3, 26], dim=-1)
y

tensor([[[[1, 2, 1],
          [3, 3, 3]],

         [[1, 1, 3],
          [2, 1, 2]]],


        [[[1, 1, 2],
          [3, 3, 3]],

         [[3, 1, 3],
          [3, 2, 2]]]])

In [12]:
y.shape, y.flatten(), y.flatten(0, 2).reshape(y.shape)

(torch.Size([2, 2, 2, 3]),
 tensor([1, 2, 1, 3, 3, 3, 1, 1, 3, 2, 1, 2, 1, 1, 2, 3, 3, 3, 3, 1, 3, 3, 2, 2]),
 tensor([[[[1, 2, 1],
           [3, 3, 3]],
 
          [[1, 1, 3],
           [2, 1, 2]]],
 
 
         [[[1, 1, 2],
           [3, 3, 3]],
 
          [[3, 1, 3],
           [3, 2, 2]]]]))

测试deep copy

In [13]:
import copy
b = y.flatten(0, -2)
c = copy.deepcopy(y.flatten(0, -2))
print(y)
c[0] = 0
print(c)
print(y)
b[0] = 0
print(b)
print(y)

tensor([[[[1, 2, 1],
          [3, 3, 3]],

         [[1, 1, 3],
          [2, 1, 2]]],


        [[[1, 1, 2],
          [3, 3, 3]],

         [[3, 1, 3],
          [3, 2, 2]]]])
tensor([[0, 0, 0],
        [3, 3, 3],
        [1, 1, 3],
        [2, 1, 2],
        [1, 1, 2],
        [3, 3, 3],
        [3, 1, 3],
        [3, 2, 2]])
tensor([[[[1, 2, 1],
          [3, 3, 3]],

         [[1, 1, 3],
          [2, 1, 2]]],


        [[[1, 1, 2],
          [3, 3, 3]],

         [[3, 1, 3],
          [3, 2, 2]]]])
tensor([[0, 0, 0],
        [3, 3, 3],
        [1, 1, 3],
        [2, 1, 2],
        [1, 1, 2],
        [3, 3, 3],
        [3, 1, 3],
        [3, 2, 2]])
tensor([[[[0, 0, 0],
          [3, 3, 3]],

         [[1, 1, 3],
          [2, 1, 2]]],


        [[[1, 1, 2],
          [3, 3, 3]],

         [[3, 1, 3],
          [3, 2, 2]]]])


测试torch.argsort：返回shape相同的tensor，选定的dim上，index从低到高为排好序的indices。

In [14]:
a = torch.randint(0, 4, (3, 4, 5))
a

tensor([[[0, 3, 3, 2, 0],
         [3, 0, 2, 2, 3],
         [1, 3, 3, 0, 3],
         [3, 3, 0, 3, 0]],

        [[1, 0, 3, 3, 0],
         [3, 0, 3, 1, 2],
         [0, 1, 0, 0, 2],
         [0, 0, 1, 0, 3]],

        [[0, 0, 1, 2, 0],
         [3, 2, 2, 3, 0],
         [2, 2, 2, 2, 2],
         [1, 3, 3, 2, 3]]])

In [15]:
indices = torch.argsort(a, dim=1, descending=True)
indices

tensor([[[1, 0, 0, 3, 1],
         [3, 2, 2, 0, 2],
         [2, 3, 1, 1, 0],
         [0, 1, 3, 2, 3]],

        [[1, 2, 0, 0, 3],
         [0, 0, 1, 1, 1],
         [2, 1, 3, 2, 2],
         [3, 3, 2, 3, 0]],

        [[1, 3, 3, 1, 3],
         [2, 1, 1, 0, 2],
         [3, 2, 2, 2, 0],
         [0, 0, 0, 3, 1]]])

不应用于训练阶段，只应用于训练完成后，应用模型进行预测的阶段。

非极大值抑制（Non-Maximum Suppression）：
- 首先，对每个grid cell负责的B个框，该box负责预测物体的置信度 * 该box负责预测物体的条件下是各个类别的条件概率
  - 获得形如$batch\_size * (S_h * S_w * B) * ClassNum$的全概率，和形如$batch\_size * (S_h * S_w * B) * 4$的center格式的2D框；
- 其次，将小于阈值的全概率置零；
- 然后，对全概率，在dim=1上做argsort，获得降序的indices；
- 最后，对每个类别，对该类别的每个mini batch，对该类别该mini batch的全概率降序的box1，对该类别该mini batch全概率小于该box1的每个box2，若box1和box2的IoU大于阈值，则将box2的全概率置零。

In [16]:
def Yolo_v1_NMS(YOLO_pred, img_shape=448, S=7, B=2, Class_Num=20, threshold_confidence=0.5, threshold_IOU=0.5):
    '''
    非极大值抑制
    - YOLO_pred: (batch_size, S_h, S_w, B*5+Class_Num)形状的张量
    - threshold_confidence: 抹去低置信度的阈值
    - threshold_IOU: 抹去高交并比的阈值
    '''
    # 参数计算
    if isinstance(img_shape, int):
        img_h = img_shape
        img_w = img_shape
    elif hasattr(img_shape, '__iter__'):
        img_h = img_shape[0]
        img_w = img_shape[1]
    ## 网格的个数形状
    if isinstance(S, int):
        S_h = S
        S_w = S
    elif hasattr(S, '__iter__'):
        S_h = S[0]
        S_w = S[1]
    grid_h = img_h / S_h     # 网格的高度和宽度
    grid_w = img_w / S_w
    
    # 在最后一维上拆分YOLO_pred
    split_list = []
    for _ in range(B):
        split_list.append(5)
    split_list.append(Class_Num)
    sp = torch.split(YOLO_pred, split_list, dim=-1)
    # 构建batch_size * (S_h * S_w * B) * (4+Class_Num)的2D框和全概率矩阵
    import copy
    mats = []
    for i in range(B):
        box = copy.deepcopy(sp[i].flatten(1, 2))           # 框
        conf = copy.deepcopy(sp[-1].flatten(1, 2))         # 预测各个类别的条件概率
        # 将2D框中心点，从相对grid cell左上角的位置的归一化，处理成图片中的位置
        for j in range(S_w):
            for k in range(S_h):
                idx = k*S_h+j
                box[:, idx, 0] = (box[:, idx, 0] + j) * grid_w      # x_c
                box[:, idx, 1] = (box[:, idx, 1] + k) * grid_h      # y_c
        box[:, :, 2] = box[:, :, 2] * img_w    # w，最大不能超过整张图片
        box[:, :, 3] = box[:, :, 3] * img_h    # h
        conf[:] = conf * box[:, :, -1].reshape(box.shape[0], box.shape[1], 1)
        mats.append(torch.concat([utils.bbox_center_to_corner(box[:, :, :-1]), conf], dim=-1))
    bboxes = torch.concat(mats, dim=1)
    
    # Non maximum supression
    prob = bboxes[:, :, 4:]
    prob[prob<threshold_confidence] = 0         # 置信度阈值筛选（prob不是copy来的，是复制了索引，所以也会改变bboxes中的值）
    indices = torch.argsort(prob, dim=1, descending=True)       # 第1维做argsort，即（S_h * S_w * B）展平后对应的维度
    for i in range(Class_Num):
        # 按类别处理
        for batch in range(prob.shape[0]):
            # 每一个小batch
            for j in range(prob.shape[1]):
                # 该batch，按第i类全概率的从大到小排列的每一个box
                idx1 = indices[batch, j, i]
                box1 = bboxes[batch, idx1, :4]
                prob1 = bboxes[batch, idx1, 4+i]
                if prob1 == 0:
                    break
                for k in range(prob.shape[1]-j-1):
                    # 位于第j个box后面的每个box
                    idx2 = indices[batch, j+k+1, i]
                    box2 = bboxes[batch, idx2, :4]
                    prob2 = bboxes[batch, idx2, 4+i]
                    if prob2 == 0:
                        break
                    # 计算两个box的iou，若大于阈值，则box2的全概率置零
                    if utils.IoU(box1, box2) > threshold_IOU:
                        bboxes[batch, idx2, 4+i] = 0
    return bboxes

In [17]:
a = torch.rand((2, 2, 2, 35))
a

tensor([[[[0.2909, 0.0566, 0.8975, 0.8648, 0.4648, 0.9205, 0.0990, 0.4620,
           0.6357, 0.8942, 0.0236, 0.6944, 0.6561, 0.8878, 0.9821, 0.6583,
           0.8177, 0.3093, 0.1912, 0.6971, 0.4892, 0.8955, 0.4803, 0.0099,
           0.2163, 0.3113, 0.1072, 0.3823, 0.5122, 0.0401, 0.1430, 0.5318,
           0.3729, 0.2996, 0.6083],
          [0.4449, 0.0971, 0.8036, 0.9549, 0.0437, 0.2797, 0.6409, 0.4622,
           0.9144, 0.7013, 0.7148, 0.3133, 0.0124, 0.1259, 0.3945, 0.7720,
           0.7258, 0.8219, 0.3268, 0.0867, 0.8560, 0.7144, 0.1042, 0.1580,
           0.3601, 0.0874, 0.7386, 0.7104, 0.9042, 0.8691, 0.2626, 0.5077,
           0.7314, 0.3285, 0.4286]],

         [[0.2068, 0.1682, 0.0337, 0.6131, 0.3510, 0.2399, 0.1452, 0.2914,
           0.3356, 0.9604, 0.2175, 0.8654, 0.3242, 0.4736, 0.9332, 0.2088,
           0.6001, 0.2149, 0.2930, 0.2897, 0.4015, 0.1565, 0.6218, 0.0914,
           0.8608, 0.3013, 0.6956, 0.8515, 0.2207, 0.9866, 0.9445, 0.1494,
           0.0937, 0.7350,

In [18]:
Yolo_v1_NMS(a, S=2, B=3).shape

torch.Size([2, 12, 24])